In [ ]:
# default_exp sparse.criteria

# Criteria

>  Which parameter is important in a neural network ?

In [ ]:
#export
import torch
import torch.nn as nn
import torch.nn.functional as F

In [ ]:
#export
def l1_norm(m, granularity):

    if granularity == 'weight':
        w = m.weight.abs()

    elif granularity == 'vector':
        dim = 3 # dim=1 -> channel vector, dim=2 -> column vector, dim=3 -> row vector
        w = (torch.norm(m.weight, p=1, dim=dim)/(m.weight.shape[3])).unsqueeze(dim) # Normalize the norm to be consistent for different dimensions

    elif granularity == 'kernel':
        w = (torch.norm(m.weight, p=1, dim=(2,3))/(m.weight.shape[2:].numel()))[:,:, None, None]

    elif granularity == 'filter':       
        w = (torch.norm(m.weight, p=1, dim=(1,2,3))/(m.weight.shape[1:].numel()))[:, None, None, None]

    else: raise NameError('Invalid Granularity')

    return w

def grad_crit(m, granularity):
    if m.weight.grad is not None:
        if granularity == 'weight':
            w = (m.weight*m.weight.grad).data.pow(2)

        elif granularity == 'vector':
            w = ((m.weight*m.weight.grad).data.pow(2).sum(dim=dim)/(m.weight.shape[3])).unsqueeze(dim)

        elif granularity == 'kernel':
            w = ((m.weight*m.weight.grad).data.pow(2).sum(dim=(2,3))/(m.weight.shape[2:].numel()))[:,:, None, None]

        elif granularity == 'filter':       
            w = ((m.weight*m.weight.grad).data.pow(2).sum(dim=(1,2,3))/(m.weight.shape[1:].numel()))[:, None, None, None]

        else: raise NameError('Invalid Granularity') 

        return w

def movement(m):
    if hasattr(m, '_old_weights') == False:
        m.register_buffer("_old_weights", m._init_weights.clone()) # If the previous value of weights is not known, take the initial value

    old_weights = getattr(m, "_init_weights")

    if granularity == 'weight': 
        w = torch.abs((m.weight.view(-1)).clone()) - torch.abs(old_weights.view(-1).clone())

    elif granularity == 'vector': 
        w = torch.abs(m.weight.sum(dim=(3)).clone()) - torch.abs(old_weights.sum(dim=(3).clone()))

    elif granularity == 'kernel': 
        w = torch.abs(m.weight.sum(dim=(2,3)).clone()) - torch.abs(old_weights.sum(dim=(2,3).clone()))           

    elif granularity == 'filter': 
        w = torch.abs(module.weight.sum(dim=(1,2,3)).clone()) - torch.abs(old_weights.sum(dim=(1,2,3).clone()))

    else: raise NameError('Invalid Granularity')

    m._old_weights = m.weight.clone() # The current value becomes the old one for the next iteration

    return w